## Springer API Simple Request Example

Example of a simple request searching for the combination Machine Learning AND (boolean) coronary in the configured metadata fields and abstracts of the articles in the database.

In [6]:
import requests

adress = 'http://api.springer.com/meta/v1/json?'
queryterms = 'q=Coronary AND Machine Learning AND year:2015'
api_key = '&api_key=f1333dfd6f52767d0a77099010fbefbc'
max_record = '&p=100'
start_record='&s=1'
year = ' year:2017'
records_type = ' type:Journal'

query = queryterms
url = adress + query + api_key + start_record + max_record

r = requests.get(url)

print("request: \n", r.url, "\n")
print("results: ", r.json()['result'])
print("keys: \n", r.json().keys())
print("\nArticle Fields:\n ", r.json().get('records')[0].keys())


request: 
 http://api.springer.com/meta/v1/json?q=Coronary%20AND%20Machine%20Learning%20AND%20year:2015&api_key=f1333dfd6f52767d0a77099010fbefbc&s=1&p=100 

results:  [{'total': '371', 'start': '1', 'pageLength': '100', 'recordsDisplayed': '100'}]
keys: 
 dict_keys(['query', 'apiKey', 'result', 'records', 'facets'])

Article Fields:
  dict_keys(['identifier', 'url', 'title', 'creators', 'publicationName', 'issn', 'eissn', 'openaccess', 'journalid', 'doi', 'publisher', 'publicationDate', 'onlineDate', 'coverDate', 'printDate', 'volume', 'number', 'issuetype', 'topicalCollection', 'startingPage', 'endingPage', 'copyright', 'genre', 'abstract'])


In [20]:
import re
import requests

class Springer_Searcher():
    
    api_key = 'f1333dfd6f52767d0a77099010fbefbc'
    address = 'http://api.springer.com/meta/v1/json?'
    
    def __init__(self, queryterms: list=None, api_key: str=None, address: str=None): 
        self.articles_found = []
        if queryterms: 
            self.queryterms = queryterms
        if api_key: 
            self.api_key = api_key
        if address: 
            self.address = address
            
    def search(self, queryterms: list=None,max_records: int=100,start_record: int=1, year: int=None):
        
        """
        @param queryterms: list of lists. Terms within the same list are
            separated by an OR. Lists are separated by an AND
        @param max_records: Number of results to return in this request.
        @param start_record: Return results starting at the number specified.
        @param year: limit to articles/chapters published in a particular year.
        """
        
        if not queryterms: 
            queryterms = self.queryterms
        
        
        
        formated_query = " AND ".join(["(%s)" % " OR ".join(term)for term in queryterms])
        
        url = self.address + "q=" + formated_query 
        
        if year:
            url += ' year:' + str(year)
        if max_records: 
            url += '&p=' + str(max_records)
        if start_record: 
            url += '&s=' + str(start_record)
        
        
        url += '&api_key=' + self.api_key
        
        r = requests.get(url)
        
        text = str(r.json().get('result'))
        lst = re.findall("'\S+'", text)
        
        print("Requisição: ")
        print("\n", r.url, "\n")
        total_records = int (lst[1][1:-1])
        print(round((total_records/max_records)+0.4999), " requests needed...")
        self.articles_found = r.json().get('records')
        print("a request completed...")
        while (start_record + max_records) < total_records:
            url = url.replace('&s='+str(start_record), 
                              '&s='+str(start_record+max_records) )
            r = requests.get(url)
            self.articles_found += r.json().get('records')
            start_record += max_records
            print("a request completed...")
        
        return self.articles_found
        

# # Using the Springer_Searcher class

In [21]:
technology_queryterms = [
    'machine learning', 'deep learning', 'artificial intelligence', 
    'neural network', 'scoring system'
]

health_queryterms = [
    'coronary artery disease', 'chest pain', 'heart disease', 'MACE', 
    'Acute Cardiac Complications'
]

queryterms = [technology_queryterms, health_queryterms]

springer_searcher = Springer_Searcher()
articles = springer_searcher.search(queryterms=queryterms)

print("\nTotal found:", len(articles))
print("\nArticle fields\n", articles[0].keys())
print("\nArticle List\n")
for i, article in enumerate(articles): 
    print(i+1, " - ", article.get('title'))

Requisição: 

 http://api.springer.com/meta/v1/json?q=(machine%20learning%20OR%20deep%20learning%20OR%20artificial%20intelligence%20OR%20neural%20network%20OR%20scoring%20system)%20AND%20(coronary%20artery%20disease%20OR%20chest%20pain%20OR%20heart%20disease%20OR%20MACE%20OR%20Acute%20Cardiac%20Complications)&p=100&s=1&api_key=f1333dfd6f52767d0a77099010fbefbc 

3  requests needed...
a request completed...
a request completed...
a request completed...

Total found: 230

Article fields
 dict_keys(['identifier', 'url', 'title', 'creators', 'publicationName', 'issn', 'eissn', 'openaccess', 'journalid', 'doi', 'publisher', 'publicationDate', 'onlineDate', 'coverDate', 'printDate', 'volume', 'number', 'issuetype', 'topicalCollection', 'startingPage', 'endingPage', 'copyright', 'genre', 'abstract'])

Article List

1  -  Proceedings of Réanimation 2018, the French Intensive Care Society International Congress
2  -  Atlas-Based Computational Analysis of Heart Shape and Function in Congenital He